## Search the Best Arbitrage in Bostrom Network

In [1]:
from subprocess import Popen, PIPE
import json
import pandas as pd
import numpy as np

POOL_FEE = 0.0015
MIN_ARBITRAGE_REVENUE = 0.02
H_START_AMOUNT = 1_000_000

#### Get Pools Balances

In [2]:
def execute_bash(bash_command: str):
    if len(bash_command.split('"')) == 1:
        _bash_command_list = bash_command.split()
    elif len(bash_command.split('"')) == 2:
        _bash_command_list = \
            bash_command.split('"')[0].split() + \
            [bash_command.split('"')[1]]
    elif len(bash_command.split('"')) > 2:
        _bash_command_list = \
            bash_command.split('"')[0].split() + \
            [bash_command.split('"')[1]] + \
            [item for items in bash_command.split('"')[2:] for item in items.split()]
    else:
        return None, f'Cannot split bash command {bash_command}'
    popen_process = Popen(_bash_command_list, stdout=PIPE)
    return popen_process.communicate(timeout=15)

def get_json_from_bash_query(bash_command: str):
    _res = execute_bash(bash_command)
    if _res[0]:
        return json.loads(_res[0].decode('utf8').replace("'", '"'))
    return

pools_json = get_json_from_bash_query('cyber query liquidity pools --node https://rpc.bostrom.cybernode.ai:443 -o json')
pools_df = pd.DataFrame(pools_json['pools'])
pools_df['balances'] = \
    pools_df['reserve_account_address'].map(
        lambda address: get_json_from_bash_query(f'cyber query bank balances {address} --node https://rpc.bostrom.cybernode.ai:443 -o json')['balances'])
pools_df

,id,type_id,reserve_coin_denoms,reserve_account_address,pool_coin_denom,balances
0,1,1,"[boot, hydrogen]",bostrom1wrtkzr96362ty7ad0qrwhkpx743xcjrtv7j2cw,pool70D7610CBA8E94B27BAD7806EBD826F5626C486BBF...,"[{'denom': 'boot', 'amount': '73435432220'}, {..."
1,2,1,"[boot, milliampere]",bostrom1y7au5el59mf8mlak53gfvmr75gr2m2jz6lra0s,pool27BBCA67F42ED27DFFB6A450966C7EA206ADAA42BA...,"[{'denom': 'boot', 'amount': '21167067005'}, {..."
2,3,1,"[boot, tocyb]",bostrom1tkpsxklqu74eqsmezcwnc5hmfswrjgn9elcneq,pool5D83035BE0E7AB904379161D3C52FB4C1C392265AC...,"[{'denom': 'boot', 'amount': '13552468565'}, {..."
3,4,1,"[hydrogen, tocyb]",bostrom1kqkwg2eq9fc5r8f0n66fj6ekc5h3kkmqhthkqx,poolB02CE42B202A71419D2F9EB4996B36C52F1B5B60DA...,"[{'denom': 'hydrogen', 'amount': '39155141169'..."
4,5,1,"[hydrogen, milliampere]",bostrom10xu73c3nkcdcf6j6u4j2kzq0cjftfj2nv7xz9y,pool79B9E8E233B61B84EA5AE564AB080FC492B4C953A9...,"[{'denom': 'hydrogen', 'amount': '235669825323..."
5,6,1,"[hydrogen, millivolt]",bostrom1u3u628ue3c5hn7ya4x4cjlk5t2d0unnarrq34k,poolE479A51F998E2979F89DA9AB897ED45A9AFE4E7DE3...,"[{'denom': 'hydrogen', 'amount': '209861712699..."


#### Calculate Prices

In [3]:
POOL_FEE = 0.0015

coins_list = list(pools_df['reserve_coin_denoms'])
coins_unique_set = set(np.concatenate(coins_list).flat)
coins_df = pd.DataFrame(columns=coins_unique_set, index=coins_unique_set)

for index, pool_row in pools_df.iterrows():
    coins_pair = pool_row.reserve_coin_denoms
    balances = {item['denom']: int(item['amount']) for item in pool_row.balances}
    coins_df.loc[coins_pair[0], coins_pair[1]] = balances[coins_pair[0]]/balances[coins_pair[1]] * (1 - POOL_FEE)
    coins_df.loc[coins_pair[1], coins_pair[0]] = balances[coins_pair[1]]/balances[coins_pair[0]] * (1 - POOL_FEE)
for coin in coins_unique_set:
    coins_df.loc[coin, coin] = 1
coins_df

,boot,millivolt,hydrogen,tocyb,milliampere
boot,1,NaN,0.585919,2.388729,13401.388502
millivolt,NaN,1,0.000009,NaN,NaN
hydrogen,1.701604,116792.129321,1,4.32843,25133.750297
tocyb,0.417378,NaN,0.230338,1,NaN
milliampere,0.000074,NaN,0.00004,NaN,1


#### Search the Best Arbitrage

In [4]:
result_list = []
for coin_1 in coins_unique_set:
    coin_1_amount = H_START_AMOUNT * coins_df.loc[coin_1, 'hydrogen']
    for coin_2 in coins_unique_set.difference({coin_1}):
        coin_2_amount = coin_1_amount * coins_df.loc[coin_2, coin_1]
        for coin_3 in coins_unique_set.difference({coin_1, coin_2}):
            coin_3_amount = coin_2_amount * coins_df.loc[coin_3, coin_2]
            result = coin_3_amount * coins_df.loc[coin_1, coin_3]
            if result > coin_1_amount * (1 + MIN_ARBITRAGE_REVENUE):
                result_list.append(
                        [[coin_1, coin_2, coin_3],
                         round(result/coin_1_amount, 3)])
            for coin_4 in coins_unique_set.difference({coin_1, coin_2, coin_3}):
                coin_4_amount = coin_3_amount * coins_df.loc[coin_4, coin_3]
                result = coin_4_amount * coins_df.loc[coin_1, coin_4]
                if result > coin_1_amount * (1 + MIN_ARBITRAGE_REVENUE):
                    result_list.append(
                        [[coin_1, coin_2, coin_3, coin_4],
                         round(result/coin_1_amount, 3)])

result_df =\
    pd.DataFrame(
        data=[[' -> '.join(coin for coin in item[0] + [item[0][0]]), item[1]] for item in result_list],
        columns=['arbitrage', 'revenue']).sort_values('revenue', ascending=False)
result_df

,arbitrage,revenue
0,boot -> milliampere -> hydrogen -> boot,1.096
3,hydrogen -> boot -> milliampere -> hydrogen,1.096
8,milliampere -> hydrogen -> boot -> milliampere,1.096
2,boot -> tocyb -> hydrogen -> boot,1.059
4,hydrogen -> boot -> tocyb -> hydrogen,1.059
7,tocyb -> hydrogen -> boot -> tocyb,1.059
1,boot -> milliampere -> hydrogen -> tocyb -> boot,1.029
5,hydrogen -> tocyb -> boot -> milliampere -> hy...,1.029
6,tocyb -> boot -> milliampere -> hydrogen -> tocyb,1.029
9,milliampere -> hydrogen -> tocyb -> boot -> mi...,1.029
